In [1]:
import sys
sys.path.append(r"../../../")
#sys.path.append(r"../utils/data_manipulation")
from app_project_utils import align_multiple_measurement_times, get_target_variables, load_data
import pandas as pd
import numpy as np
from random import randint


In [2]:
age = 19
groups = ['control']
df = load_data(group='all')
target_variables = get_target_variables()
df = align_multiple_measurement_times(df, target_variables, age, groups)


# Initialize Q-table
q_table = {}

In [ ]:
class Env:
    def __init__(self, df, patient_id):
        self.df = df
        self.patient_id = patient_id
        
    def get_value(self, action):
        value = df.query("patient_id == patient_id")[action.key]
        return value
    
    def do_action(self, state, action):
        value = self.get_value(action)
        next_state = state.get_next_state(action, value)

In [ ]:
class Action:
    def __init__(self, key):
        self.key = key

    

True

In [ ]:
class State:
    def __init__(self, state:dict):
        self.state = state
    
    def is_equal(self, other_state:dict):
        # dict == dict: key and values comparsion
        if self.state == other_state.state:
            return True
        else:
            return False
        
    def get_next_state(self, action, value):
        new_state_dict = self.state.copy()
        new_state_dict[action.key] = value
        return State(new_state_dict)
    
    
    def get_possible_actions(self):
        possible_actions = []
        for key in self.state.keys():
            if self.state[key] is None:
                possible_actions.append(key)
        return possible_actions

In [3]:

# Define rewards
REWARD_CORRECT_PREDICTION = 10
REWARD_ASKING_INFORMATIVE_QUESTION = 1
REWARD_ASKING_REDUNDANT_QUESTION = -1

# Define discount factor and learning rate
GAMMA = 0.9
ALPHA = 0.1



In [16]:
def choose_action(state):
    # Choose a random action if no entries in Q-table for this state
    if state not in q_table.keys():
        return randint(0, len(df) - 1)
    
    # Choose the action with highest Q-value
    return max(q_table[state], key=q_table[state].get)

In [17]:
def update_q_table(state, action, reward, next_state):
    # Calculate the Q-value update
    q_value = q_table.get(state, {}).get(action, 0)
    q_value += ALPHA * (reward + GAMMA * max(q_table.get(next_state, {}).values())) - q_value
    q_table[state][action] = q_value

In [18]:
# Define a simple prediction function based on average target for questions asked
def predict(state):
    return np.mean([df.iloc[action]['target'] for action in state])


In [19]:
def run_simulation(df, max_questions):
    total_reward = 0
    init_state = State({feature: None for feature in feats})
    current_state =  init_state
    for _ in range(max_questions):
        action = choose_action(state)
        # Get features and target for chosen question
        question_features = df.iloc[action].drop('target')
        question_target = df.iloc[action]['target']
        # Update state with new answer
        state[action] = question_features

        # Calculate reward based on prediction and informativeness
        reward = REWARD_CORRECT_PREDICTION if question_target == predict(state) else 0
        reward += REWARD_ASKING_INFORMATIVE_QUESTION if question_features.sum() > 0 else REWARD_ASKING_REDUNDANT_QUESTION

        # Update Q-table
        update_q_table(state.copy(), action, reward, state)
        total_reward += reward

    return total_reward, state

In [20]:

# Run simulations and print results
for _ in range(10):
    reward, _ = run_simulation(df.copy(), 5)
    print(f"Total reward: {reward}")



{}


TypeError: unhashable type: 'dict'